# MB-Fit Documentation (v1.0)

## Introduction

Over the last decades, the branch of computational chemistry has played an important role in the study of energetic, dynamical and structural properties of chemical and physical processes such as vibrational spectra, phase diagrams, isomerization processes, chemical reactions... Computational chemistry provides a powerful tool to gain insights on the driving forces of these processes under the assumption that the energies, or more accurately, the energy differences, are correct.

When exploring the potential energy surface (PES), there are several computational methods available. *Ab initio* methods provide an accurate solution, depending on the method itself and the basis set used, but the treatable system size is usually small, from a few atoms when using coupled-cluster theory, to a few hundreds of atoms when using density functional theory (DFT). These methods solve the Schrodinger Equation

$$\hat{H}\Psi = E\Psi$$

for wavefunction methods such as Hartree-Fock, perturbation theory, or coupled-cluster, or the Kohn-Sham equations

$$\left( -\frac{\hbar ^2}{2m} \nabla ^2 + v_{eff}(\vec{r})\right) = \varepsilon _i \varphi _i (\vec{r})$$

$$\rho(\vec{r}) = \sum _i ^N \left\lvert\varphi _i (\vec{r}) \right\rvert ^2 \label{eq:dens}$$

in the case of DFT. Commonly, systems of interest can have thousands of atoms, and exploring the PES with *ab initio* methods is virtually impossible nowadays. However, one can go to the force field (FF) realm to find very simplified potential energy functions (PEFs) typically based on harmonic potentials for bonded interactions, Lennard-Jones potentials for non-bonded interactions, and point-charge electrostatic interactions. 

$$ $$

These PEFs enable the treatment of tens of thousands of atoms, and can be further simplified by coarse-graining them to reach hundreds of thousands of atoms. However, the accuracy of FFs is not great. We can plot these different methods in a plot of size vs. accuracy.

<img src="figures/plot_methods.png" alt="plotmethods" width="500"/>

Ideally, we would like to be in the predictive representation area: have a great accuracy and be able to treat large system sizes. Several approaches can be taken, and one of them is based on the many-body expansion (MBE) of the energy of a system, which can be decomposed as the sum of the one-body energy, plut the two body energy, plus the three-body energy, and so on, until the n-th body:

$$ E_{\text{N}}(1,\dots,N) = \sum_{i=1}^N V^{\text{1B}}(i) + \sum_{i<j}^NV^{\text{2B}}(i,j) 
	+ \sum_{i<j<k}^N V^{\text{3B}}(i,j,k) + \dots + V^{\text{NB}}(1,\dots,N) $$

where $V^{\text{1B}}(i) = 0$ and $V^{\text{1B}}(i) = E(i) - E_{\text{eq}}(i)$ for atomic and molecular monomers, respectively. 
In the latter case, $V^{\text{1B}}(i)$ corresponds to the one-body (1B) energy required to deform an individual monomer from its equilibrium geometry, and $V^{\text{nB}}$ are the $n$-body (nB) energies defined recursively as

$$ V^{\text{nB}}(1,\dots,n) = E_n(1,\dots,n) - \sum_iV^{\text{1B}}(i) - \sum_{i<j}V^{\text{2B}}(i,j) - \sum_{i<j<\dots<n-1}V^{\text{(n-1)B}}(i,j,\dots,(n-1)) $$

Since the MBE converges quickly for non-metallic systems such as CH$_4$ and H$_2$O, it provides a rigorous and efficient framework for the development of full-dimensional PEFs in which each individual term of the MBE can be separately determined from high-level electronic structure calculations. 

This notebook provides with the infrastructure to generate one- and two-body PEFs for small molecules, i.e. molecules that have a maximum bond distance between atoms of 4, such as ethane, carbon dioxide, or the case we will present in this document, ammonia. The different PEFs, functional form and terms will be presented as they appear. 

## Requirements

Add the requirements here

## Getting started

This notebook will serve as a template that one can use for other small molecules. The first step to ensure that the library can be imported is to add the path to the MB-Fit library to the Python path. To do so, first define the `MBFIT_HOME` environment variable to point to the folder where `MB-fit` is located, and source the file `mbfit.sh` located inside `MBFIT_HOME`. If this step has not been done before opening the notebook, please copy these lines in a bash terminal and reopen the notebook.

```sh
export MBFIT_HOME="PATH/TO/MBFIT"
source ${MBFIT_HOME}/mbfit.sh
```

These two lines of code will enable access to the `mbfit` module from any Python script. Now the library should be imported without any error.

In [ ]:
import potential_fitting as mbfit

## One-body PEF

Between 2013 and 2014, the Paesani lab published a full dimentional many-body PEF for water. MB-pol is based on the many-body expansion of the total energy of a system, and  it uses different expressions for the different many-body terms. The one-body term was borrowed from the PEF developed by Partridge and . This surface is fitted to reproduce energies from ab initio data, experimental rotational constants, vibrational spectra, and other properties. I was shown to be an excellent model to describe the behavior of an isolated water molecule. From now on, when we talk about MB-pol, $V^{\text{1B}} = V^{\text{1B}}_{\text{PS}}$

The one-body energy is also known as the distorion energy, and is typically described by force fields using harmonic fucntions for bonds and angles, and other simple functional forms for dihedrals and inversion angles. Within the MB-nrg framework, the distortion energy will be described with PIPs. The goal of this section is to obtain $V^{\text{1B}}(\vec{r})$ Contrariely as for MB-pol, we will develop a PEF for any other molecule, since there are not too many surfaces in the literature with similar accuracy as the one for water.

The functional form that MB-nrg uses is a set of permutationally invariant polynomials that are a function of the internal distances of the atoms in the monomer.

$$ V^{\text{1B}} = \sum _i A_i P[\xi _a  \xi _b \dots \xi _n] $$

$A_i$ is a linear constant, and P is a set of permutations that make the system permutationally invariant with respect to the variables $\xi _j$ that are present in the element $i$ of the sum. The polynomial variables $\xi$ are a function of a distance between two atoms $r$, and can have different functional forms with the condition that they must decay to 0 when $r \rightarrow \infty$. Examples of possible acceptable variable functional forms are Morse type variables ($\xi(r) = e^{-kr}$) or Coulomb type variables ($\xi(r) = e^{-kr}/r$)

This next part of this text will show how to generate the necessary data to obtain these linear ($A_i$) and non-linear ($k_j$) parameters that allow the best description of the reference PES.

### Electronic structure calculation details

MB-Fit supports two different software packages to perform calculations: `QChem` and `Psi4`. There is no difference in using one or the other in terms of obtaining the PEF. For this example we will use `Psi4`, along with the DFT functional $\omega$B97M-V and the aug-cc-pvtz basis set. All this information will be stored in variables.

#### Electronic structure software

The options for the package are `"qchem"` to use QChem, and `"psi4"` to use Psi4. 

In [ ]:
code = "qchem"

#### Electronic structure method

The options here are any method that the ES software can handle. The most common methods to use are any supported DFT functional and coupled-cluster theory (CCSD(T)):
```
method = "HF"     # will use Hartree-Fock
method = "MP2"    # will use the Moller-Plesset perturbation theory of order 2
method = "PBE0"   # will use the DFT functional PBE0
```
In this example we will use $\omega$B97M-V. This hybrid functional has been shown to have coupled-cluster accuracy for one- and two-body interactions.

In [ ]:
method = "wb97m-v"

#### Basis set

The size of the basis set will determine how reliable are the calculated energies. A larger basis set will be closer to the limit of the level of theory used, but it will be also more computationally expensive. For DFT, aug-cc-pvtz is close to the complete basis set limit.

In [ ]:
basis = "aug-cc-pvtz"

#### Computation details and extra options

Usually, an electronic structure package needs to have the memory and the number of threads that will be used. This can be specified with the variables `num_threads` and `memory`. 

In [ ]:
# Number of threads to use
num_threads = 2

# Amount of memory to use
memory = "4GB"

A part from the basic optiones defined above, sometimes it is needed to define extra variables or parameters in the electronic structure calculation, such as number of SCF iterations, number of geometry optimization steps, energy thresholds... These options can be passed as a dictionary to the corresponding calculations. Each option has a keyword in the corresponding electronic structure software. For example, in QCHEM, to increase the number of SCF iterations, we need to define the keyword `MAX_SCF_CYCLES`, and to increase the maximum number of optimization steps, the keyword `GEOM_OPT_MAX_CYCLES`. We can add these options into a dictionarythat we will then pass to the calculation routines:
```
qm_options = {"MAX_SCF_CYCLES" : 100, "GEOM_OPT_MAX_CYCLES" : 80}
```
which would be equivalent to do:
```
qm_options["MAX_SCF_CYCLES"] = 100
qm_options["GEOM_OPT_MAX_CYCLES"] = 80
```
In general, no matter the electronic structure code that is being used, the procedure to add extra options to the input is the same: `qm_options["option"] = value`. To see the list of possible `options`, look at the documentation of the corresponding electronic structure code. In our case, we will use the following options:

In [ ]:
qchem_qm_options = {}
qchem_qm_options["purecart"] = "1111"
qchem_qm_options["xc_grid"] = "000099000590"
qchem_qm_options["nl_grid"] = "1"
qchem_qm_options["unrestricted"] = "false"
qchem_qm_options["incdft"] = "0"
qchem_qm_options["incfock"] = "0"
qchem_qm_options["max_scf_cycles"] = "100"
qchem_qm_options["scf_guess"] = "sad"
qchem_qm_options["scf_convergence"] = "6"
qchem_qm_options["thresh"] = "11"
qchem_qm_options["symmetry"] = "false" 
qchem_qm_options["sym_ignore"] = "true"
qchem_qm_options["geom_opt_max_cycles"] = 80

psi4_qm_options = {}

### System details

MB-Fit needs to know about the system we are dealing with, and the calculation details specified earlier. All this information is stored in a file with all these settings. For convenience, we generate this file *in situ*, and we describe below each one of the components that this file requires. We will refer to this file as `settings` file from now on.

#### Names

The names of the different monomers present in the system is used to identify a given monomer in the database. They have no other purpose rather than identification. The name can have any length, and combine numbers and letters. It is not required but it is recommended to use capital letters only. The `names` variable is a list of strings that contains the monomers in the system:
```
names = ["MON1","MON2",...]
```
After the definition of this list, the order of the monomers is set. If one has more than one monomer in that list, **monomer 1** will be the first element of the list, **monomer 2** the second one, and so on. When defining the properties of each monomer, this order will be important. In this first exampleit doesn't matter because there is only one monomer.

In [ ]:
names = ["NH3"]

#### Number of atoms, spin, and charge

The number of atoms, charge of each monomer, and spin multiplicity of each monomer will be defined in the lists `number_of_atoms`, `charges`, and `spin`, respectively. These lists are lists of integers, and each element corresponds to the value of the list property of the monomer in that position (as defined in the list `names`.

- `number_of_atoms` is the total number of atoms of the monomers. For example, CO$_2$ has 3 atoms, and NH$_3$ has 4. Any virtual site (either for the polynomials or for the electrostatics) should not be counted as "atom".
- `charges` is the total charge of every monomer. Neutral monomers will have a charge of 0, while ions such as Cl$^-$ or NH$_4^+$ will have a charge of -1 and 1 respectively.
- `spin` contains the spin multiplicities of the monomers, calculated as $M = 2S + 1$. For now, only closed shell systems are recommended since they have been throughly tested.

If there is only one monomer, these variables still need to be a list, with a single element, as shown below.

In [ ]:
# Number of atoms of each monomer
number_of_atoms = [4]

# Charge of each monomer
charges = [0]

# Spin multiplicity of each monomer
spin = [1]

#### Use MB-pol for monomer

As previously mentioned, MB-nrg and TTM-nrg are compatible with MB-pol by construction. When developing a two-body surface for a dimer in which one of the monomers is water, the partridge-shwenke dipole moment PEF should be used to describe the electrostatic properties of water in order to enable full compatibility with MB-pol. This can be passed to the MB-Fit library through the variable `use_mbpol`. If the value of this variable at the position `i` is 1, the monomer `i` will be assumed to be water, and the charges, polarizabilities, and polarizability factors will be calculated using the PS PEF. In this example, we just have ammonia, which is not water, and consequently, the variable `use_mbpol` will be a list of one element set to 0.

In [ ]:
use_mbpol = [0]

#### Symmetry

Commonly, in force fields, the interactions are described in a pairwise way and the different constants of the function describing an interaction depends on the atom types involved. The symmetry keyowrd in `MB-Fit` refers to the label that the atoms have. Each chemically unique atom will have a letter assigned, not related to their chemical identity. In order to build the symmetry, these are the steps to follow:
1. Write the empirical formula of your molecule. As an example, for ammonia, is `NH3`
2. Expand the chemical formula. This means that if there are two or more atoms that have a subindex, just repeat that atom. In the case of ammonia, we write `NHHH`
3. Identify which atoms are chemically equivalent. This is very simple in small molecules, but can be complciated in larger molecules. To figure this out:
   1. Locate two atoms that have the same atomic symbol. Two atoms that have different atomic symbols cannot be equivalent.
   2. Replace one atom by `X` and look at the molecule.
   3. Replace the other atom by `X` and compare this structure with the previous one. If the structure is the same, the two atoms are equivalent.
4. Replace all the atomic symbols by letters, starting from `A`, and moving up in the alphabet (`B`,`C`...) using all capital letters. Each unique atom type must be represented by a different letter. In the case of ammonia, `NHHH` will become `ABBB`
5. Compress the new label. There must be a number after each type, even if it is `1`. In the case of ammonia will be `A1B3`

Then, the symmetry list can be defined as in below.

In [ ]:
symmetry = ["A1B3"]

Once symmetry is defined, the molecule symmetry can be obtained by joining all the elements in the symmetry list with an underscore `_`. We will refer to it as `molecule_in`. In the case of ammonia, symmetry and molecule in will be the same, with the exception that molecule in is a string and not a list.

In [ ]:
molecule_in_nh3 = "_".join(symmetry)

#### SMILES string

The SMILES string is the way the connectivity of the molecule is passed into `MB-Fit`. In classical force fileds, the interactions between atoms are usually divided in bonded and non-bonded interactions. Typically, two atoms will be bonded if there are three or less bonds of distance between those two atoms, and non-bonded otherwise. When two atoms within the same molecule are non-bonded, the non-bonded interactions sucha as repulsion, dispersion and electrostatics are calculated, and ignored or scaled if the two atoms are bonded. The SMILES string will allow `MB-Fit` to find the connectivity matrix of the molecule.

The SMILES string for each fragment must follow the [general rules](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system#Description) of SMILES strings in addition to the following extra rules:
* Order of atoms in the SMILES string must match the order of the atoms in the XYZ file.
* All hydrogen atoms must be included explicity.
* Do not include any information about charges in your SMILES string.

Writting a SMILES string by scratch might seem complicated, but these steps show how to do it in a simple way.
1. Write out all atoms of the fragment in the same order they appear in your XYZ file. Remember, if any atom name is 2 letters, you must put square brackets around it.
2. Add a '.' between any atoms that are not bonded.
3. Add numbers after atoms to signify bonds between non-adjacent atoms. In the SMILES format you can bond two non-adjacent atoms by putting the same digit after them. For example, the SMILES string 'C1CC1' species a ring of 3 carbons. When there are multiple digits after a single atom each digit is treated independently. For instance, 'C12.C23.C31' also specifies a ring of 3 carbons. In this representation we have removed the implicit bond between adjacent atoms by adding a dot, and then introduced bonds between each pair of carbon atoms. If you need to specify more than 10 bonds you can use % signs to combine multiple digits: 'C%10%20.C%20%30.C%30%10' also specifies a ring of 3 carbons.

Here is an example:

Lets say our '.xyz' file for formaldehyde looks like this:
```
4
comment line
C   x   y   z
O   x   y   z
H   x   y   z
H   x   y   z
```

Step 1) COHH
Step 2) CO.H.H
Step 3) C12O.H1.H2

Here is another example:
```
3
Chloroform
C   x   y   z
H   x   y   z
Cl  x   y   z
Cl  x   y   z
Cl  x   y   z
```

Step 1) CH[Cl][Cl][Cl]
Step 2) CH.[Cl].[Cl].[Cl]
Step 3) C123H.[Cl]1.[Cl]2.[Cl]3

In the case of ammonia, assuming that the XYZ is:
```
4
comment line
N   x   y   z
H   x   y   z
H   x   y   z
H   x   y   z
```

Step 1) NHHH
Step 2) NH.H.H
Step 3) N12H.H1.H2

In [ ]:
smiles = ["N12H.H1.H2"]

### Electronic structure outputs

All the electronic structure calculations outpus will be stored. The path where they will be put is defined in `log_path`.

In [ ]:
log_path = "logs"

### The SETTINGS file

The properties of the system, along with with some of the details for the electronic structure calculations, are stored in the `settings` file. This file is passed to the different methods in `MB-Fit` so the code can retrieve the information of the system. Below there is an example of the  settings file. Any line that starts with `#` will be ignored.

```
[files]
# Local path directory to write log files in
log_path = logs

[config_generator]
# what library to use for geometry optimization and normal mode generation
code = psi4
# use geometric or linear progression for T and A in config generation, exactly 1 must be True
geometric = False
linear = False

[energy_calculator]
# what library to use for energy calculations
code = psi4

[psi4]
# memory to use when doing a psi4 calculation
memory = 4GB
# number of threads to use when executing a psi4 calculation
num_threads = 2

[qchem]
# number of threads to use when executing a qchem calculation
num_threads = 2

[molecule]
# name of fragments, seperated by commas
names = NH3
# number of atoms in each fragment, seperated by commas
fragments = 4
# charge of each fragment, seperated by commas
charges = 0
# spin multiplicity of each fragment, seperated by commas
spins = 1
# tag when putting geometries into database
tag = none
# Use or not MB-pol
use_mbpol = 0
# symmetry of each fragment, seperated by commas
symmetry = A1B3
SMILES = N12H.H1.H2
```

The user can write the file manually, or can use the following command to write the file with all the information inputed earlier.

In [ ]:
# Settings for monomer
settings_nh3 = "settings_nh3_monomer.ini"

my_settings_file = """
[files]
# Local path directory to write log files in
log_path = """ + log_path + """

[config_generator]
# what library to use for geometry optimization and normal mode generation
code = """ + code + """
# use geometric or linear progression for T and A in config generation, exactly 1 must be True
geometric = False
linear = False

[energy_calculator]
# what library to use for energy calculations
code = """ + code + """

[psi4]
# memory to use when doing a psi4 calculation
memory = """ + memory + """
# number of threads to use when executing a psi4 calculation
num_threads = """ + str(num_threads) + """

[qchem]
# number of threads to use when executing a qchem calculation
num_threads = """ + str(num_threads) + """

[molecule]
# name of fragments, seperated by commas
names = """ + names[0] + """
# number of atoms in each fragment, seperated by commas
fragments = """ + str(number_of_atoms[0]) + """
# charge of each fragment, seperated by commas
charges = """ + str(charges[0]) + """
# spin multiplicity of each fragment, seperated by commas
spins = """ + str(spin[0]) + """
# tag when putting geometries into database
tag = none
# Use or not MB-pol
use_mbpol = """ + str(use_mbpol[0]) + """
# symmetry of each fragment, seperated by commas
symmetry = """ + symmetry[0] + """
SMILES = """ + smiles[0] + """
"""

In [ ]:
# Write the file:
ff = open(settings_nh3,'w')
ff.write(my_settings_file)
ff.close()

### Polynomial Generation

The permutationally invariant polynomials can be generated using `MB-Fit`. The first step is to define all the variables that will be involved in the polynomials. They can be obtained by calling the `generate_poly_input` function, which usage is the following, and it generates a file that lists all the different variables that can be in the polynomial.

```
generate_poly_input(settings_path, molecule_in, in_file_path, virtual_sites=['X', 'Y', 'Z'])
    Generates an input file for polynomial generation.
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        molecule_in         - String idicating symmetry of molecule, ie "A1B2_A1B2" for (CO2)2
        in_file_path        - Local path to the file to write the polynomial input to.
        virtual_sites       - List of Symmetry labels that are virtual sites.
                Default: ["X", "Y", "Z"]
    
    Returns:
        None.
```

This function requires the settings file, previously defined and explained. It also requires the molecule symmetry, which as mentioned earlier, has been defined in `molecule_in`. The argument `in_file_path` will contain the name of the file that will be generated with all the information. The optional argument `virtual_sites` specifies which letters will represent virtual sites in the molecules. Currently, `MB-fit` only accepts virtual sites for `MB-pol` water, and will be described later in the manual. Before generating the polynomial input file, let's define the name:

In [ ]:
poly_in_nh3 = "poly_nh3.in"

At this point the polynomial generation input with all the variables can be generated:

In [ ]:
mbfit.generate_poly_input(settings_nh3, molecule_in_nh3, poly_in_nh3)

The file generated should be like this:
```
add_molecule['A1B3']

add_variable['A', '1', 'a', 'B', '1', 'a', 'x-intra-A+B-1']
add_variable['A', '1', 'a', 'B', '2', 'a', 'x-intra-A+B-1']
add_variable['A', '1', 'a', 'B', '3', 'a', 'x-intra-A+B-1']
add_variable['B', '1', 'a', 'B', '2', 'a', 'x-intra-B+B-1']
add_variable['B', '1', 'a', 'B', '3', 'a', 'x-intra-B+B-1']
add_variable['B', '2', 'a', 'B', '3', 'a', 'x-intra-B+B-1']
```

There are two keywords in the generated file. The first one, `add_molecule` states that a new molecule is added to the system. The second one, `add_variable`, specifies that a variable $\xi _{ij}$ that depends on the distance between atoms $i$ and $j$ will be added to the system. Concretely,
```
add_variable['A', '1', 'a', 'B', '2', 'a', 'x-intra-A+B-1']
```
translates into add a variable between the first atom of type `A` from the first monomer (`a`) `['A', '1', 'a']` and the second atom of type `B` of the first monomer (`a`) `['B', '2', 'a']`, which non-linear parameter constant will have the tag `x-intra-A+B-1`. Variables with the same constant tag will have the same non-linear parameters.

The user is free to remove any variable that is not wanted in the polynomial expression, but the use of all the variables and the addition of filters is the recommended option. When the molecule grows in size and the polynomial maximum order increases, the number of terms that the polynomial has increases exponentially. Typically, the number of terms in the polynomial should not exceed 2000, and this number is easily exceeded for larger systems. In these situations, the best solution is to apply filters. An extended explanation of the filter format and their options can be found in the `FILTERS.md` file inside `MBFIT_HOME/docs`.

Once the input and the filters have been decided, the polynomial generation function `generate_polynomials` can be called:
```
generate_polynomials(settings_path, poly_in_path, order, poly_dir_path, generate_direct_gradients=True, num_gradient_terms_per_line=1)
    Generates polynomial input for maple and some ".cpp" and ".h" polynomial files.
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        poly_in_path        - Local path to the file to read the polynomial input from. Name of file should be in
                the format "A1B2.in". It is ok to have extra directories prior to file name. For example:
                "thisplace/thatplace/A3.in".
        order               - The order of the polynomial to generate.
        poly_dir_path       - Local path to the directory to write the polynomial files in.
        generate_direct_gradients - If True, then a gradients cpp file is generate additionally to the polynomial
                cpp file. This may take a while.
                defualt: False.
        num_gradient_terms_per_line - The number of terms to put in each line in the cpp file
                for computing the gradients. Larger numbers may result in slower compilation times.
    
    Returns:
        None.
```


`settings_path` is the settings file. Since it was generated in the working directory, the name of the file will be enough, and has been defined earlier under the variable `settings_nh3`; same with the `poly_in_path`. The maximum order that the polynomial will have is set by `order`. Given a triatomic molecule `B--A--B`, there are three distances: between atoms A and B$_1$, which we call $\vec{r}_{AB_1}$; between atoms A and B$_2$ ($\vec{r}_{AB_2}$); and between atoms B$_1$ and B$_2$ ($\vec{r}_{B_1B_2}$). Consequently, there are three possible variables that can be used to generate the polynomials: $\xi _{AB_1}$, $\xi _{AB_2}$, and $\xi _{B_1B_2}$. Note that at this point the functional form of $\xi$ is still irrelevant, but it is a function of the distance between the two atoms involved. With this three variablesthe following polynomial can be written:

\begin{equation}
\begin{split}
P(\xi _{AB_1},\xi _{AB_2},\xi _{B_1B_2}) &=  K_1\xi _{AB_1} + K_2\xi _{AB_2} + K_3\xi _{B_1B_2} \leftarrow \text{Degree 1} \\
&+ K_4\xi _{AB_1}\xi _{AB_1} + K_5\xi _{AB_2}\xi _{AB_2} + K_6\xi _{B_1B_2}\xi _{B_1B_2} + K_7\xi _{AB_1}\xi _{AB_2} + K_8\xi _{AB_1}\xi _{B_1B_2} + K_9\xi _{AB_2}\xi _{B_1B_2} \leftarrow \text{Degree 2} \\
&+ K_{10}\xi _{AB_1}\xi _{AB_1}\xi _{AB_1} + ... \leftarrow \text{Degree 3}
\end{split}
\end{equation}

$P(\xi _{AB_1},\xi _{AB_2},\xi _{B_1B_2})$ can be written up to any degree, but the higher the degree, the higher the number of terms. For a given molecule, several orders should be tested and use a maximum polynomial degree $n$ that keeps the number of linear terms $K_i\xi_1 ^{a_1} ... \xi _k ^{a_k}$, with $\sum _{j=1} ^ k a_j \leq n$, lower than 2000.

In [ ]:
polynomial_order_nh3 = 2

We have been talking about permutationally invariant polynomials, but the polynomial above is not permutationally invariant. Specifically, let's look at the first degree terms $K_1\xi _{AB_1}$ and $K_2\xi _{AB_2}$. $\xi _{AB_1}$ refers to the distance between A and the **first** B atom, while $\xi _{AB_2}$ refers to the distance between A and the **second** B atom. However, both variables $\xi _{AB_1}$ and $\xi _{AB_2}$ should have the same constant parameters since both are describing the same exact distance type. If the constants $K_1$ and $K_2$ are different, changing the order of the B aoms would change the value of the polynomial evaluation, and physically, this is not correct, since no matter which order do the B atoms have, the molecule AB2 is exactly the same. The symmetrization process, which is also performed in the generation of the polynomials, fixes this issue. Instead of having two terms $K_1\xi _{AB_1}$ and $K_2\xi _{AB_2}$, the algorithm will symetrize them into $K_{\alpha}(\xi _{AB_1} + \xi _{AB_2})$. With this new term it doesnt matter which atom B is B$_1$ or B$_2$, and it will always evaluate to the same value no matter the order of identical atoms. This process is performed for all the terms that can be symmetrized, reducing the number of linear constants $K$ that will need to be fitted, and ensuring the permutationally invariance of the polynomials.

The `generate_polynomials` function can aso generate the analytical gradients of the polynomials. However, for fitting purposes, the gradients are not used. These files will be required by `MBX` in order to run molecular dynamics simulations or other calculations that require the gradients. If thegradients are generated, depending on the size of the molecule and the degree, each one of the variable gradients can have hundreds or thousands of terms. Compilation time in these situations becomes extremely slow, and can be improved with a payback on the runtime speed by splitting the calculation of each variable gradient. The number of terms per group is controlled by the `num_gradient_terms_per_line` argument. Since the purpose of this manual is to show document the fitting part, the gradients won't be generated. All the polynomial files will be generated and added to the folder defined in `poly_in_path`. 

In [ ]:
polynomial_directory_nh3 = "polynomials_files_directory_nh3_monomer"

In [ ]:
mbfit.generate_polynomials(settings_nh3, poly_in_nh3, polynomial_order_nh3, polynomial_directory_nh3, generate_direct_gradients=False)

Several files should have been generated in the folder `polynomial_directory_nh3`:
- `poly.log` contains the information about how many variables, permutations, and terms for each degree the polynomial has, including the total number of terms and the filtered terms. The total number of terms at the end of that file is the number of $K_i\xi_1 ^{a_1} ... \xi _k ^{a_k}$ terms that the polynomial will have. If this number is larger than 2000, some filtering should be added, or the maximum degree should be decreased. Physical intuition should be used when adding filters. As an example, if a given degree $n$ yields too many terms, filtering the terms that contain a high degree of a variable involving two atoms which interaction is very small, such as two hydrogen atoms, is a good strategy. This reduces the number of terms and maintains the flexibility of the polynomial.
- `vars.cpp` is a file that summarizes the variables involved in the polynomial. This file is not used by the user and contains the same information as the polynomial input file.
- `poly-direct.cpp` is a C++ function that given the linear coefficients $K_i$ and the variables $\xi _j$ will return the evaluation of the polynomial at that point. A `poly-direct-grd.cpp` file will be generated if the gradients have been calculated. This file will return the same value as the `poly-direct.cpp` file in addition to the gradients of each one of the derivatives. Summarizing, given $\vec{K} = [K_1, K_2, ..., K_M]$ and $\vec{\xi} = [\xi _1,\xi _2, ..., \xi _N]$, where $M$ is the number of terms and $N$ is the number of variables, `poly-direct.cpp` will return $V_{poly}$, and `poly-direct-grd.cpp` will return the same $V_{poly}$ and the gradients $\nabla P(\xi _1, ... , \xi _N) = \lbrack \frac{\partial P}{\xi _1},..., \frac{\partial P}{\xi _N} \rbrack$. The `poly-model.h` file is the header file for both gradient and no-gradient files.
- `poly-nogrd.maple` and  `poly-grd.maple` are files that are automatically generated no matter the options in the polynomials. The polynomial evaluations in the direct form is slow, and if the user has MAPLE, the function `execute_maple` can be used to optimize the evaluation of the polynomials. This step will generate optimized evaluations for the polynomials with and without gradients. 

```
execute_maple(settings_path, poly_dir_path)
    Runs maple on the polynomial files in the specified directory to turn them into actual cpp files.
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        poly_directory      - Local path to the directory to read the ".maple" files from and write the ".cpp" files
                to.
    
    Returns:
        None.
```

The two arguments of this function have been previously described, and it will generate two extra useful files to be used for the energy evaluation function. 

In [ ]:
mbfit.execute_maple(settings_nh3, polynomial_directory_nh3)

If the previous command has been executed, two extra files conaining the optimized evaluation of the polynomial function with and without gradients have been generated, being `poly-grd.cpp` and `poly-nogrd.cpp` those files. **Maybe talk about optimization...?**

### Training set and test set generation

In any machine learning scheme, the goal is to be able to predict the outcome of an event by using knowledge of events in which the outcome is known. In the case of energy, the goal is to predict the energy of a molecular system given the coordinates of all the atoms. We need to train the model with configurations for which the energy is known. These set of configurations will be the **training set**. To assess that the model actually predicts the right energies, we will also need a set of configurations for which the energy is known, but that will not be included in the training set. This set of configurations will be called the **test set**. Usually we will assess the accuracy of a fit by calculating the root mean square deviation (RMSD) of the training and test sets. Ideally, both sets should have a low RMSD and of a cimilar order of magnitude. To calculate the RMSD, the following expression is used:
$$\text{RMSD} = \sqrt{\frac{\sum _{i=1} ^N (y_i^{calc} - y_i^{ref})^2}{N}}$$
where $N$ is the number of data points, $y_i^{calc}$ is the value of the predicted energy, and $y_i^{ref}$ is the reference energy of that same configuration.

The degree of the polynomial and the size of the training set must be carefully selected to avoid underfitting and overfitting. Underfitting occurs when the function used as model is not flexible enough to reproduce the reference data. An example of underfitting is found on the left panel in the figure below. The model used is a straight line, and it is trying to reproduce a curve with not much success. Although this is a problem in terms of accuracy of the model, it usually does not represent a problem in its stability. However, when the model has too many parameters in comparison with the number of points in the training set, overfitting can occur. The fit will have an apparent excellent agreement with the reference data, but the test set will probably have a bad agreement with the model. When the RMSD of the training set is small, but the one for the test set is large, overfitting is most likely happening and we need to either increase the size of our training set, or decrease the number of terms. When overfitting occurs, there is a big chance of finding **holes** in the PEF, which means that the models does not have information in a region of the space, and the energy that is predicted is usually largely negative, as it is shown in the right pannel of the figure below. Ideally, the wanted behavior is the one shown in the central pannel, in which the model behaves properly within the range of the sample points.

<img src="figures/over_underfitting.png" alt="overunderfitting" width="1000"/>

We recommend to have a training set with a number of configurations that is 20 times the number of linear terms in the polynomials, with a minimum of 500 configurations, and use a 20% of that number for the test set. We first need to define the names of the training and the test sets. First we will generate the configurations, and then we will generate the formatted training set.

In [ ]:
# XYZ file with the configurations of the training set
training_configs_nh3 = "training_configs_nh3.xyz"

# XYZ file with the configurations of the test set
test_configs_nh3 = "test_configs_nh3.xyz"

# XYZ file with the training set that the codes need to perform the fit
# Configurations are the same as training_configs but this file
# has the energies in the comment line
training_set_nh3 = "training_set_nh3.xyz"

# XYZ file with the test set that the codes need to perform the fit
# Configurations are the same as test_configs but this file
# has the energies in the comment line 
test_set_nh3 = "test_set_nh3.xyz"

After the definition of the filenames that will contain the training and test configurations and formatted sets, we have to specify the details of the training set. The number of configuration will be the first point to tackle. As mentioned, it should be at least 20 times the number of terms:

In [ ]:
# Number of configurations in the 1b training_set
num_training_configs = 500

# Number of configurations in the 1b test set
num_test_configs = 100

The training set and the test set must contain configurations around the minimum energy structure (the configuration with the lowest energy). However, distorted configurations must also appear to avoid extrapolationg values for the polynomials. The average energy, following a Boltzman distribution, in a molecular dynamics simulation at room temperature is 0.593 kcal/mol. Technically, having distortions up to 0.6 kcal/mol should be enough to have a stable simulation at room temperature. However, 0.593 kcal/mol is the average energy that a molecule will have, and does not mean that a higher values are not possible. There might be instantaneous fluctuations in which the molecule is distorted several kcal/mol, and if those configurations are not sampled in the training set, the PEF will have a hole, giving a non-physical value for the energy. The recomendation is to use a maximum distortion energy ($E_i ^{dist} = E_i -E_i ^{min}$) of 100 kcal/mol. This will ensure that the configurational space that can be visited up to several hundreds Kelvin is represented in the training and the test set.

In [ ]:
# Maximum energy allowed for distorted monomers (in kcal/mol)
maximum_1b_energy = 100.0

The structure of the formatted training set is nothing else than a single XYZ file with all the configurations, one after the other one, with two numbers in the comment line, which are, in this order, the binding energy and the n-body energy. In this first ammonia example, the n-body energy is the one-body energy. The different orders of the many-body energies have been introduced earlier. However, the concept binding energy (BE) appears here for the first time. There are multiple definitions on how to calculate the BE, but the simplest way is:
$$ \text{BE} = E^{tot} - \sum _{i=1} ^N E_i ^{opt}$$
where $E^{tot}$ is the total energy of the system, $N$ is the total number of monomers in the system, and $E_i^{opt}$ is the optimized energy of monomer $i$. If we work out the equation for one-body, we conclude that for this case, the binding energy and the distorsion energy are the same.
$$ \text{BE} = E^{tot} - E_1 ^{opt} = E ^{dist}$$
Thus, the maximum binding energy will be set to be the same as the maximum distortion energy (or also one-body energy).

In [ ]:
maximum_binding_energy = maximum_1b_energy

The training set and the test set will both be generated, in the case of the one-body PEF, using normal mode samplig. To do so, the first step is to optimize the molecule and calculate the normal modes. We first need to define a structure that is close the the minimum energy structure.

In [ ]:
# XYZ file that contains the unoptimized geommetry of monomer 1
unoptimized_nh3 = "nh3.xyz"

my_unopt_monomer = """4
unoptimized nh3
 N                 -0.79953653    0.30706836    0.00000000
 H                 -0.46621464   -0.63574472    0.00000000
 H                 -0.46619743    0.77846854    0.81649673
 H                 -0.46619743    0.77846854   -0.81649673
"""

In [ ]:
# Write the file:
ff = open(unoptimized_nh3,'w')
ff.write(my_unopt_monomer)
ff.close()

The XYZ file where the optimized monomer structure will be sotred has to be defined too.

In [ ]:
optimized_nh3 = "nh3_opt.xyz"

The next step is to perform the optimization. The function `optimize_geometry` will take care of that. 

```
optimize_geometry(settings_path, unopt_geo_path, opt_geo_path, method, basis, qm_options={})
    Optimizes the geometry of the given molecule.
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        unopt_geo_path      - Local path to the file to read the unoptimized geoemtry from.
        opt_geo_path        - Local path to the file to write the optimized geometry to.
        method              - The method to use for this geometry optimization.
        basis               - The basis to use for this geometry optimization.
        qm_options           - Dictionary of extra arguments to be passed to the QM code doing the calculation.
    
    Returns:
        None.
```

All the arguments of this function have been previously defined. After the call, a new file with the optimized geometry will have appeared in the working directory. This function will use the electronic structure method defined in `method` with the basis set defined in `basis`, to optimize the geometry in the file defined by `unoptimized_nh3`, and save the result in the file defined by `optimized_nh3` using the options defined in `qm_options`.

In [ ]:
mbfit.optimize_geometry(settings_nh3, unoptimized_nh3, optimized_nh3, method, basis, qchem_qm_options)

Once the optimized structure has been obtained, it can be used to calculate the normal modes with the `generate_normal_modes` function, which will perform a normal mode calculation using the software, method and basis set specified.

```
generate_normal_modes(settings_path, opt_geo_path, normal_modes_path, method, basis, qm_options={})
    Generates the normal modes for the given molecule.
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        opt_geo_path        - Local path to the file to read the optimized geometry from.
        normal_modes_path   - Local path to the file to write the normal modes to.
        method              - The method to use for this normal modes calculation.
        basis               - The basis to use for this normal modes calculation.
        qm_options           - Dictionary of extra arguments to be passed to the QM code doing the calculation.
    
    Returns:
        Null dimension of normal modes.
```

The normals modes will be calculated, and their frequencies and displacement vectors will be stored in the formatted file `normal_modes_path`. 

In [ ]:
normal_modes_nh3 = "normal_modes_nh3.dat"

In [ ]:
mbfit.generate_normal_modes(settings_nh3, optimized_nh3, normal_modes_nh3, method, basis, qchem_qm_options)

**Explanation of the normal mode generation training set.**
- Explain the method
- explain the different distributions (A dist vs T dist (What is A? do we need it?), classical, quantum,..)
- explain the default splittting of the temperatures

Using the methodology previously describe, we can generate configurations using the function `generate_normal_mode_configurations`:

```
generate_normal_mode_configurations(settings_path, opt_geo_path, normal_modes_path, configurations_path, number_of_configs=100, seed=None, linear=False, geometric=False, temperature=None, classical=True, temp_distribution=None, A_distribution=None)
    Generates normal mode configurations for a given molecule from a set of normal modes.
    
    If both linear and geometric are False, will use a piecewise distribution over temperature.
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        opt_geo_path        - Local path to the file to read optimized geometry from.
        normal_modes_path   - Local path to the file to read normal modes from.
        config_path         - Local path to the file to write configurations to.
        number_of_configs   - Number of configurations to generate
        seed                - The same seed with the same molecule and normal modes will always generate the same
                configurations.
        linear              - If True, then use a linear distribution over temp and A.
                    Default: False
        geometric           - If True, then use a geometric distribution over temp and A.
                    Default: False
        temperature         - Temperature at which normal mode sampling is done. If specified, configurations
                will use clasical normal mode distribution at the specified temperature instead of either geometric
                or linear progression.
        classical           - If True, use a classical distribution over temp and A, otherwise, use a quantum
                distribution. QM distributions generate a wider distribution over energy.
                Default: True
        temp_distribution   - Implementation of DistributionFunction. If specified, then the temperature
                distribution specified by the linear, geometric, or temperature arguments will be ignored and this
                distribution will be used instead. Should be implemented over the domain [0,1]. So the first config
                will have temperature = temp_distribution.get_value(0) and the last config will have temperature =
                temp_distribution.get_value(1).
        A_distribution      - Implementation of DistributionFunction. If specified, then the A
                distribution specified by the linear, geometric, or temperature arguments will be ignored and this
                distribution will be used instead. Should be implemented over the domain [0,1]. So the first config
                will have A = A_distribution.get_value(0) and the last config will have A =
                A_distribution.get_value(1).
    
    Returns:
        None.
```

In this function, some of the arguments have already been seen. The configurations that will be generated are going to be stored in the file defined by `config_path`, previoulsy defined as `training_configs_nh3` and `test_configs_nh3`, and will generate as many configurations as `number_of_configs` indicates. 

The selection of the normal mode coordinates will be given by random numbers. As in any random number generator, it needs a seed. If no seed is given to the functions that use random numbers, the machine time will be taken as seed, which ensures different results every time that the fnction is run. The recomendation is to use a defined seed for every function call, to ensure reproducibility of the results. However, the seeds for different sets of the training or test set must be different, or exactly the same configurations will be generated. The seed must be an integer within the python integer range:

($2^{31} - 1$,$-(2^{31}) - 2$)

In [ ]:
seed_training = 12345
seed_test = 54321

The distribution that will be used must also be defined. The distribution used is going to be over temperatures, but using the predefined percentages at each percentage of the maximum temperature. In order to indicate that, the following variables must be defined:

In [ ]:
linear = False
geometric = False
temperature = None

Using or not a classical distribution is the user's choice. Classical distributions will result in narrower distributions, but at high temperature of sampling, the classical and the quantum distribution will become more similar. In this example, the classical distribution will be used.

In [ ]:
classical = True

**What is tem_dist and A_dist**

In [ ]:
temperature_distribution = None
A_distribution = None

All the options have been defined, and the training and test set configurations can be now generated:

In [ ]:
# Training Set
mbfit.generate_normal_mode_configurations(settings_nh3, optimized_nh3, normal_modes_nh3, 
                                          training_configs_nh3, number_of_configs=num_training_configs, 
                                          seed=seed_training, linear=linear, geometric=geometric, 
                                          temperature=temperature, classical=classical, 
                                          temp_distribution=temperature_distribution, 
                                          A_distribution=A_distribution)

#Test Set
mbfit.generate_normal_mode_configurations(settings_nh3, optimized_nh3, normal_modes_nh3, 
                                          test_configs_nh3, number_of_configs=num_test_configs, 
                                          seed=seed_test, linear=linear, geometric=geometric, 
                                          temperature=temperature, classical=classical, 
                                          temp_distribution=temperature_distribution, 
                                          A_distribution=A_distribution)

With the configurations generated, the next step is to calculate the energy of each one of them. `MB-Fit` provides with an infrastructure based on POSTGRE database. 

**Maybe add some info about what to do and how to install**

After set up and installation of the database, the information to access it must be written in a file. Any function that performs operations on the database, either writting or reading information, needs a `database.ini` file with the information about the host, the port, the database name, the username, and the password. The format of this file is similar to the `settings` file:

```
[database]
host = host.name.edu
port = 5432
database = my_database
username = my_username
password = my_password
```

The file is created below:

In [ ]:
database_settings = "database.ini"
my_database_settings = """[database]
host = piggy.pl.ucsd.edu
port = 5432
database = potential_fitting
username = potential_fitting
password = 9t8ARDuN2Wy49VtMOrcJyHtOzyKhkiId
"""

# Write the file. Remember to update the username and password!
ff = open(database_settings,'w')
ff.write(my_database_settings)
ff.close()

Once the database is set up, the configurations generated for the test and for the training set can be added with the function `init_database`. This will add all the configurations of a given XYZ file to the database. If the configuration is already there and it has the same tag, it won't add it. 

```
init_database(settings_path, database_config_path, configurations_path, method, basis, cp, *tags, optimized=False)
    Creates a database from the given configuration .xyz files. Can be called on a new database
    to create a new database, or an existing database to add more energies to be calculated
    
    Args:
        settings_path       - Local path to the file containing all relevent settings information.
        database_config_path - .ini file containing host, port, database, username, and password.
                    Make sure only you have access to this file or your password will be compromised!
        configurations_path - Local path to a single .xyz file.
        method              - QM method to use to calculate the energy of these configurations.
        basis               - QM basis to use to calculate the energy of these configurations.
        cp                  - Use counterpoise correction for these configurations?
        tags                - Mark the new configurations with these tags.
        optimized           - Are these configurations optimized geometries? Defualt is False.
    
    Returns:
        None.
```

`settings_path` is the path to the settings file as usual, and `database_config_path` is the file that contains the database access information. `configurations_path` is the XYZ file that contains the configuration(s) that need to be added to the database. Each file needs to be added in a different function call. `method` and `basis` are the electronic structure method and basis set that will be used to calculate the energies of the configurations added. When using small basis sets, and even large basis sets that are not close to the complete basis set limit, it is recomended to use counter-poise correction to correct for the basis set superposition error (BSSE). This option can be activated by setting `cp` to `True`. This correction only takes effect if there is more than one monomer, so for the one-body calculations can be set to `False`.

In [ ]:
cp = False

After these mandatory arguments, the `tags` are needed. The function of the tags is to differenciate molecules that are the same but belong to multiple sets. In this simple example, there are two sets that should have excluding geometries, i.e. a geometry in the training set should not be present in the test set and viceversa. Two tags will be used: `"training_nh3"` for the training set and `"test_nh3"` for the test set. Finally, the `optimized` argument indicates if the configurations added are optimized (the minimum energy structure) or not, and can be set to `True` or `False`. The optimized geommetry must always be in any set, since is used to calculate the binding energy required to generate the formatted training sets. To add the configurations:

In [ ]:
# Add training set configurations
mbfit.init_database(settings_nh3, database_settings, training_configs_nh3, method, basis, cp, "training_nh3", optimized = False)

# Add test set configurations
mbfit.init_database(settings_nh3, database_settings, test_configs_nh3, method, basis, cp, "test_nh3", optimized = False)

In [ ]:
# Add the optimized monomer to both tags
mbfit.init_database(settings_nh3, database_settings, optimized_nh3, method, basis, cp, "training_nh3", "test_nh3", optimized = True)

In [ ]:


# IDs of the monomers (should be consistent with the 1B id for each)
mon_ids = ["co2"]

# Number of TTM-nrg fits to perform
num_mb_fits = 5

In [ ]:
# PostgreSQL database that stores structures and energies
database_config = "local.ini"
client_name = "motzu the survivor"

In [ ]:
# Configuration file that contains all the monomer 
# and dimer information. Will be used to generate the 2B codes.
config = "config.ini"

In [ ]:
# Directory where mb-nrg fitting code will be stored
mbnrg_directory = "mb-nrg_fit"
mbnrg_fit_path = "fits_mbnrg"

### 1.4. Training and test set generation

#### 1.4.2. Add configurations to the database

#### 1.4.3. Calculate energy

In [ ]:
help(mbfit.fill_database)

In [ ]:
potential_fitting.fill_database(mon_settings, database_config, client_name, "train_example1_co2", "test_example1_co2", calculation_count = None)

#### 1.4.4. Training set and Test set generation

In [ ]:
help(potential_fitting.generate_training_set)

In [ ]:
# Generate training set
potential_fitting.generate_training_set(mon_settings, database_config, training_set, method, basis, cp, "train_example1_co2", e_bind_max = bind_emax, e_mon_max = mon_emax)

# Generate test set
potential_fitting.generate_training_set(mon_settings, database_config, test_set, method, basis, cp, "test_example1_co2", e_bind_max = bind_emax, e_mon_max = mon_emax)

### 1.5. Obtain charges, polarizabilities, and C6

In [ ]:
help(potential_fitting.get_system_properties)

In [ ]:
chg, pol, c6 = potential_fitting.get_system_properties(mon_settings, config, geo_paths = [opt_mon])

In [ ]:
help(potential_fitting.write_config_file)

In [ ]:
potential_fitting.write_config_file(mon_settings, config, chg, pol, [opt_mon], c6)

### 1.6. MB-nrg fit

#### 1.6.1. Obtain and compile the fitting code

In [ ]:
help(potential_fitting.generate_mbnrg_fitting_code)

In [ ]:
potential_fitting.generate_mbnrg_fitting_code(mon_settings, config, poly_in, poly_directory, polynomial_order, mbnrg_directory, use_direct=False)

In [ ]:
help(potential_fitting.compile_fit_code)

In [ ]:
potential_fitting.compile_fit_code(mon_settings, mbnrg_directory)

### 1.6.2. Perform the fit

In [ ]:
help(potential_fitting.prepare_fits)

In [ ]:
potential_fitting.prepare_fits(mon_settings, mbnrg_directory, 
                               training_set, mbnrg_fit_path, 
                               DE=20, alpha=0.0005, num_fits=num_mb_fits, 
                               ttm=False, over_ttm=False)

In [ ]:
help(potential_fitting.execute_fits)

In [ ]:
potential_fitting.execute_fits(mon_settings,mbnrg_fit_path)

In [ ]:
help(potential_fitting.retrieve_best_fit)

In [ ]:
potential_fitting.retrieve_best_fit(mon_settings, mbnrg_fit_path, fitted_nc_path = "mbnrg.nc")

### 1.7. Visualize the results

In [ ]:
help(potential_fitting.get_correlation_data)

In [ ]:
energies = potential_fitting.get_correlation_data(mon_settings, mbnrg_directory, mbnrg_fit_path,
                                       test_set, split_energy = 5.0)

### 1.8 Add files to MBX

In [ ]:
help(potential_fitting.generate_MBX_files)

In [ ]:
potential_fitting.generate_MBX_files(mon_settings, config, mon_ids, 
                                     do_ttmnrg=False, mbnrg_fits_path=mbnrg_fit_path,  
                                     MBX_HOME = None, version = "v1")